In [1]:
# 악성문장을 인식하는 기능

In [1]:
from happytransformer import HappyTextClassification

/Users/jongholee/opt/anaconda3/envs/py38_nlp/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
happy_tc = HappyTextClassification("BERT", "Hate-speech-CNERG/dehatebert-mono-english", 2)

Downloading: 100%|██████████████████████████| 1.23k/1.23k [00:00<00:00, 486kB/s]
Downloading: 100%|███████████████████████████| 669M/669M [00:30<00:00, 21.7MB/s]
Downloading: 100%|█████████████████████████████| 152/152 [00:00<00:00, 45.3kB/s]
Downloading: 100%|████████████████████████████| 872k/872k [00:01<00:00, 617kB/s]
Downloading: 100%|█████████████████████████████| 112/112 [00:00<00:00, 27.3kB/s]
10/08/2022 12:08:38 - INFO - happytransformer.happy_transformer -   Using model: cpu


In [3]:
result = happy_tc.classify_text("GPT-2 was created by OpenAI")
print(result)
print(result.label)
print(result.score)

TextClassificationResult(label='NON_HATE', score=0.9717797636985779)
NON_HATE
0.9717797636985779


In [4]:
result = happy_tc.classify_text("Good job Bert!")
print(result)

TextClassificationResult(label='NON_HATE', score=0.9751752614974976)


In [5]:
result = happy_tc.classify_text("It's a disappointing story. The more I read, the more I feel like throwing up. Seem foolish.")
print(result)

TextClassificationResult(label='NON_HATE', score=0.9709519147872925)


In [6]:
result = happy_tc.classify_text("fucking ridiculous bullshit, fuck you")
print(result)

TextClassificationResult(label='HATE', score=0.5498354434967041)


In [7]:
result = happy_tc.classify_text("!&#*#&!*@&#*@*!*(#*")
print(result)

TextClassificationResult(label='HATE', score=0.520082414150238)


In [9]:
# 다른 모델 적용 테스트

In [8]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

ckpt = 'Narrativa/byt5-base-tweet-hate-detection'

tokenizer = AutoTokenizer.from_pretrained(ckpt)
model = T5ForConditionalGeneration.from_pretrained(ckpt).to("cuda")

def classify_tweet(tweet):

    inputs = tokenizer([tweet], padding='max_length', truncation=True, max_length=512, return_tensors='pt')
    input_ids = inputs.input_ids.to('cuda')
    attention_mask = inputs.attention_mask.to('cuda')
    output = model.generate(input_ids, attention_mask=attention_mask)
    return tokenizer.decode(output[0], skip_special_tokens=True)
    
    
classify_tweet('fucking ridiculous bullshit')

Downloading: 100%|██████████████████████████| 2.85k/2.85k [00:00<00:00, 623kB/s]
Downloading: 100%|██████████████████████████████| 711/711 [00:00<00:00, 140kB/s]
Downloading: 100%|██████████████████████████| 2.50k/2.50k [00:00<00:00, 707kB/s]
Downloading: 100%|█████████████████████████| 2.33G/2.33G [01:44<00:00, 22.2MB/s]


AssertionError: Torch not compiled with CUDA enabled

In [9]:
# cuda -> cpu로 변경


In [10]:
# 잘 작동함

from transformers import AutoTokenizer, T5ForConditionalGeneration

ckpt = 'Narrativa/byt5-base-tweet-hate-detection'

tokenizer = AutoTokenizer.from_pretrained(ckpt)
model = T5ForConditionalGeneration.from_pretrained(ckpt).to("cpu")

def classify_comment(tweet):

    inputs = tokenizer([tweet], padding='max_length', truncation=True, max_length=512, return_tensors='pt')
    input_ids = inputs.input_ids.to('cpu')
    attention_mask = inputs.attention_mask.to('cpu')
    output = model.generate(input_ids, attention_mask=attention_mask)
    return tokenizer.decode(output[0], skip_special_tokens=True)
    
    
classify_comment('fucking ridiculous bullshit')

/Users/jongholee/opt/anaconda3/envs/py38_nlp/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'hate-speech'